<a href="https://colab.research.google.com/github/aditya27you/MEME/blob/main/task1_movies_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .appName("MoviesAnalysis") \
    .getOrCreate()
